Pulizia actors.csv e caricamento in PostGres

In [94]:
import pandas as pd
import os
import re
from unidecode import unidecode

In [95]:
# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "actors.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Crea la cartella clean_data se non esite
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)


In [96]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [97]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisce i valori nulli con "Unknown" o il valore più comune
df['role'] = df['role'].fillna("Unknown")
df['name'] = df['name'].fillna("Unknown")

In [98]:
# Funzione per tradurre il nome in caratteri latini 
def transliterate_name(name):
    if isinstance(name, str):  # Controlla che sia una stringa
        return unidecode(name).strip()  # Traduci e rimuovi spazi extra
    return name  # Se non è stringa, lascialo così

# Applica la funzione alla colonna dei nomi e dei ruoli
df['name'] = df['name'].apply(transliterate_name)
df['role'] = df['role'].apply(transliterate_name)


In [99]:
# Prende solo la prima parte prima di "/"
df['role'] = df['role'].str.split('/').str[0]

# Rimuove sinonimi tra parentesi
df['role'] = df['role'].apply(lambda x: re.sub(r"\(.*?\)|（.*?）", "", x))

# Standardizza il testo (se ci sono nomi di attori, ruoli, ecc.)
df['name'] = df['name'].str.strip().str.title()
df['role'] = df['role'].str.strip().str.title()

print(df['role'].unique())

['Barbie' 'Ken' 'Gloria' ... 'An Du' 'Ba Cai' 'Zuo Yila']


In [100]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "actors_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

Caricamento file csv pulito nel database con PostgreSQL


In [103]:
import psycopg2
from sqlalchemy import create_engine,text

In [109]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"  
DB_USER = "postgres"       
DB_PASSWORD = "2004agnese"   
DB_HOST = "localhost"      
DB_PORT = "5432"    

conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database
cur.execute(f"CREATE DATABASE {DB_NAME};") # Esegue il comando per creare il database con il nome specificato

In [110]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [111]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS actors_data (
        id INT,
        name TEXT,
        role TEXT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [112]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("actors_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

502